# 1. Input Austria Data 

In [1]:
from spglm.iwls import _compute_betas_gwr
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from copy import deepcopy

/opt/conda/lib/python3.8/site-packages/spglm/utils.py:367: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if resetlist is not ():


In [2]:
austria = pd.read_csv('Data/austria.csv')
austria = austria[austria['Origin'] != austria['Destination']]

flows = austria['Data'].values 
Oi = austria['Oi'].values 
Wj = austria['Dj'].values 
Dij = austria['Dij'].values 
Origin = austria['Origin'].values 
Destination = austria['Destination'].values

X = austria.copy()
X['Code'] = X['Origin'] + X['Destination']
# the unnamed column store id which will be repalced by 'Code' column
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
X = X[['Code','Origin', 'Destination', 'Data', 'Oi', 'Dj', 'Dij']] 

# 2. PoissonGAM -- Local Scoring Algorithm

In [3]:
'''
The codes are written by Dr.Taylor Oshan
'''
def local_score(y, X, verbose=False):
    s_0 = np.log(np.mean(y))
    eta = s_0.reshape((-1, 1))
    s_old = np.zeros_like(X)
    crit = 9999
    
    while crit > 1e-8:
        mu = np.exp(eta).reshape((-1, 1))
        w = mu.reshape((-1,1))
        z = eta + ((y.reshape((-1, 1)) - mu) / mu)
        betas = backfit(z, X, w, verbose=verbose)
        s_new = np.multiply(betas.T, X)
        inner = np.sum((s_old - s_new)**2, axis=1)
        num = np.sum(w*inner)
        den = np.sum(w*np.sum((1 + s_old)**2, axis=1).reshape((-1, 1)))
        crit = num / den
        eta = np.sum(s_new, axis=1).reshape((-1, 1))
        s_old = s_new
    return betas, eta


# In[817]:

# backfitting - inner loop
def backfit(y, X, w, verbose=False):
    n,k = X.shape
    betas = _compute_betas_gwr(y, X, w.reshape((-1, 1)))[0]
    XB = np.multiply(betas.T, X)
    yhat = np.dot(X, betas)
    err = y.reshape((-1, 1)) - yhat
    iters = 0
    scores = []
    delta = 1e6
    tol = 1e-8
    max_iter = 50

    for iters in range(1, max_iter + 1):
        new_XB = np.zeros_like(X)
        params = np.zeros_like(betas)

        for j in range(k):
            temp_y = XB[:, j].reshape((-1, 1))
            temp_y = temp_y + err.reshape((-1, 1))
            temp_X = X[:, j].reshape((-1, 1))

            beta = _compute_betas_gwr(temp_y, temp_X, w.reshape((-1, 1)))[0]
            #print(beta)
            yhat = np.dot(temp_X, beta)
            new_XB[:, j] = yhat.flatten()
            err = (temp_y - yhat).reshape((-1, 1))
            params[j, :] = beta[0][0]
    
        num = np.sum((XB-new_XB)**2)
        den = 1 + np.sum(np.sum(XB, axis=1)**2)
        score = (num / den)
        XB = new_XB
        
        scores.append(deepcopy(score))
        delta = score

        if verbose:
            print("Current iteration:", iters, ",SOC:", np.round(score, 8))
        if delta < tol:
            break

    return params

# 3. Accessibility in Competing Destination Model

## 3.1. Setup

In [4]:
def setup_Aij(X_df):
    
    '''
    return:
        a new dataframe with two new columns storing `mk` and `dkj` arrays with respect to each flow
    ------------------------------------------------------------------------------------------------
    We can calculate Aij based on the equation `Aij = sum(mk * (dkj ** sigma))`; 
    The implement is shown below:

        X['Aij'] = ((X['mk']* (X['dkj'] ** sigma)).apply(lambda x: sum(x).item()))
    
    '''
    
    X = X_df.copy()

    for temp_or in X['Origin']: 
    
        #find all flows whose origin is temp_or
        temp_df = X[X['Origin'] == temp_or]
    
        #all destinations whose origin is temp_or
        temp_j = temp_df['Destination'].values
    
        #calculate aij for each destination temp_de
        for temp_de in temp_j:
        
            #each flow's code
            temp_code = temp_or + temp_de
        
            #alternative destination with respect to temp_de
            alters = temp_j[temp_j != temp_de]
        
            #find distances between temp_de and all other alternative destinations and attractiveness of temp_de
            temp_df2 = X[X['Origin'] == temp_de]
            temp_df3 = temp_df2[temp_df2['Destination'].isin(alters)]
        
            mk = temp_df3['Dj'].values.reshape((-1,1))
            dkj = temp_df3['Dij'].values.reshape((-1,1))

            # aij = sum(mk * (dkj ** sigma))
            
            # create a list containing mk and dkj repectively
            
            X.loc[X['Code'] == temp_code, 'mk'] = X['Code'].apply(lambda x: mk if x == temp_code else None)
            X.loc[X['Code'] == temp_code, 'dkj'] = X['Code'].apply(lambda x: dkj if x == temp_code else None)

    return X

In [5]:
X_new = setup_Aij(X)

### An example of calculating Aij based on mk and dkj columns

In [7]:
sigma = -1
X_ex = X_new.copy()
X_ex['Aij'] = ((X_ex['mk'] * (X_ex['dkj'] ** sigma)).apply(lambda x: sum(x).item()))

X_ex = X_ex[['Code','Origin', 'Destination', 'Data', 'Oi', 'Dj', 'Dij', 'Aij' , 'mk', 'dkj']] 
X_ex.head()

,Code,Origin,Destination,Data,Oi,Dj,Dij,Aij,mk,dkj
1,AT11AT12,AT11,AT12,1131,4016,25741,103.001845,772.885445,"[[26980], [4117], [8634], [8193], [4902], [395...","[[45.796272], [216.994739], [129.878172], [140..."
2,AT11AT13,AT11,AT13,1887,4016,26980,84.204666,711.035103,"[[25741], [4117], [8634], [8193], [4902], [395...","[[45.796272], [249.932874], [158.630661], [186..."
3,AT11AT21,AT11,AT21,69,4016,4117,220.811933,453.279306,"[[25741], [26980], [8634], [8193], [4902], [39...","[[216.994739], [249.932874], [92.407958], [151..."
4,AT11AT22,AT11,AT22,738,4016,8634,132.007480,538.803623,"[[25741], [26980], [4117], [8193], [4902], [39...","[[129.878172], [158.630661], [92.407958], [124..."
5,AT11AT31,AT11,AT31,98,4016,8193,214.511814,509.093317,"[[25741], [26980], [4117], [8634], [4902], [39...","[[140.706671], [186.420738], [151.777157], [12..."


## 3.2. Calculation 

In [8]:
def calculate_Aij(X_df, sigma):
    '''
    for the convenience of finding optimal σ 
    '''
    X = X_df.copy()
    X['Aij'] = ((X['mk']* (X['dkj'] ** sigma)).apply(lambda x: sum(x).item()))
    X = X[['Code','Origin', 'Destination', 'Data', 'Oi', 'Dj', 'Dij', 'Aij' , 'mk', 'dkj']] 
    X.head()
    
    return X  

# 4. Implement of PoissonGAM in CD Model 

## 4.1. Find optimal σ

In [9]:
def spAccess (X, y, w, sigma_min = -4, sigma_max = 4, step = 0.1, tol = 1e-8, max_iter = 100):
    
    def aic_func(sigma, X = X, y = y, w = w):
        
        #calculate the Aij
        Aij = calculate_Aij(X, sigma)['Aij'].values
        ln_Aij = np.log(Aij).reshape((-1,1))
        #temp_x = np.multiply(w.reshape((-1, 1)) ,ln_Aij)
        
        #Xn = np.hstack((np.log(X['Oi'].values).reshape((-1,1)), np.log(X['Dj'].values).reshape((-1,1)), np.log(X['Dij'].values).reshape((-1,1)), ln_Aij))
        #temp_x =  w * Xn
        
        mod = OLS(y, ln_Aij).fit()
        #mod = sm.GLM(y, sm.add_constant(temp_x), family=sm.families.Poisson()).fit()
        #print(mod.aic)
        return mod.aic
    
    def find_opt_sigma(sigma_a = sigma_min, sigma_c = sigma_max, step = step, tol = tol):
        
        sigma_b = sigma_min + step * np.abs(sigma_max - sigma_min)
        sigma_d = sigma_max - step * np.abs(sigma_max - sigma_min)
        
        diff = np.inf
        optimal_sigma = np.inf
        iters = 1
    
        while (diff > tol and iters <= max_iter):
            
            aic_b = aic_func(sigma_b)
            aic_d = aic_func(sigma_d)
            
            if aic_b <= aic_d: #discard the old c, b<d<c
                optimal_sigma = sigma_b
                sigma_c = sigma_d
                sigma_d = sigma_b
                sigma_b = sigma_a + step * np.abs(sigma_c - sigma_a)
            
            else: # discard the old a, a<b<d
                optimal_sigma = sigma_d
                sigma_a = sigma_b
                sigma_b = sigma_d
                sigma_d = sigma_c - step * np.abs(sigma_c - sigma_a)
            
            diff = np.abs(aic_b - aic_d)      
            optimal_sigma = np.round(optimal_sigma, 3)
            iters +=1

        return optimal_sigma
    
    optimal_sigma = find_opt_sigma()
    #print(optimal_sigma, aic_func(optimal_sigma))
    Aij = calculate_Aij(X, optimal_sigma)['Aij'].values
    ln_Aij = np.log(Aij).reshape((-1,1))
    
    #beta = _compute_betas_gwr(y, ln_Aij, w.reshape((-1, 1)))[0]
    
    return ln_Aij, optimal_sigma #, beta

## 4.2. Local Scoring Algorithm with Golden Section Search

In [67]:
def local_score2(y, X, verbose=False):
    s_0 = np.log(np.mean(y))
    eta = s_0.reshape((-1, 1))
    crit = 9999
    sigma = 0
    ln_Aij_init = calculate_Aij(X, -1)['Aij'].values.reshape((-1,1))
    
    # with intercept
    # Xn = np.hstack((np.ones(72).reshape((-1,1)), np.log(X['Oi'].values).reshape((-1,1)), np.log(X['Dj'].values).reshape((-1,1)), np.log(X['Dij'].values).reshape((-1,1))))
    Xn = np.hstack((np.log(X['Oi'].values).reshape((-1,1)), np.log(X['Dj'].values).reshape((-1,1)), np.log(X['Dij'].values).reshape((-1,1))))
    X_ = np.hstack((Xn, ln_Aij_init)) 
    s_old = np.zeros_like(X_) 
    
    while crit > 1e-8:
        mu = np.exp(eta).reshape((-1, 1))
        w = mu.reshape((-1,1))
        z = eta + ((y.reshape((-1, 1)) - mu) / mu)
        
        betas, ln_Aij, sigma = backfit2(z, X_, X, w, verbose=verbose)
        X_ = np.hstack((Xn, ln_Aij))
        s_new = np.multiply(betas.T, X_)
        
        inner = np.sum((s_old - s_new)**2, axis=1)
        num = np.sum(w*inner)
        den = np.sum(w*np.sum((1 + s_old)**2, axis=1).reshape((-1, 1)))
        crit = num/den
        eta = np.sum(s_new, axis=1).reshape((-1, 1))
        s_old = s_new
    
    return betas, sigma, eta

# backfitting and golden section search - inner loop
def backfit2(y, Xn, X, w, verbose=False):
    n,k = Xn.shape  
    betas = _compute_betas_gwr(y, Xn, w.reshape((-1, 1)))[0]
    XB = np.multiply(betas.T, Xn)
    
    yhat = np.dot(Xn, betas)
    err = y.reshape((-1, 1)) - yhat
    iters = 0
    scores = []
    delta = 1e6
    tol = 1e-8
    max_iter = 50
    optimal_sigma = 0
    ln_Aij = 0
    
    for iters in range(1, max_iter + 1):
        new_XB = np.zeros_like(Xn)
        params = np.zeros_like(betas) 

        for j in range(k-1):# the additive component with respect to Aij is not here
            temp_y = XB[:, j].reshape((-1, 1))
            temp_y = temp_y + err.reshape((-1, 1)) 
            temp_X = Xn[:, j].reshape((-1, 1))

            beta = _compute_betas_gwr(temp_y, temp_X, w.reshape((-1, 1)))[0]
            print(beta)
            
            yhat = np.dot(temp_X, beta)
            new_XB[:, j] = yhat.flatten()
            err = (temp_y - yhat).reshape((-1, 1))
            params[j] = beta
            
        temp_y = XB[:, -1].reshape((-1,1)) + err.reshape((-1, 1))
        ln_Aij, optimal_sigma = spAccess(X, temp_y, w)
        beta = _compute_betas_gwr(temp_y, ln_Aij, w.reshape((-1, 1)))[0]
        params[-1] = beta   
        print(beta)
        print(optimal_sigma)
        
        yhat = np.dot(ln_Aij, beta)
        new_XB[:, -1] = yhat.flatten()
        err = (temp_y - yhat).reshape((-1, 1))
        
        num = np.sum((XB-new_XB)**2)
        den = 1 + np.sum(np.sum(XB, axis=1)**2)
        score = (num / den)
        XB = new_XB
        
        scores.append(deepcopy(score))
        delta = score

        if verbose:
            print("Current iteration:", iters, ",SOC:", np.round(score, 8))
        if delta < tol:   
            break
    
    #additive component Aij with the optimal sigma value in each iteration

    #print(optimal_sigma)

    return params, ln_Aij, optimal_sigma

In [68]:
coefs, sigma, eta = local_score2(flows, X_new, True)

[[0.00264212]]
[[2.0960456]]
[[-1.41506991]]
[[-0.11989132]]
4.0
Current iteration: 1 ,SOC: 0.0530213
[[0.00625448]]
[[2.07618342]]
[[-1.39471242]]
[[-0.11879864]]
4.0
Current iteration: 2 ,SOC: 0.00080523
[[0.00990111]]
[[2.05655166]]
[[-1.37500674]]
[[-0.11767313]]
4.0
Current iteration: 3 ,SOC: 0.0007826
[[0.01358045]]
[[2.03714861]]
[[-1.35594023]]
[[-0.11651592]]
4.0
Current iteration: 4 ,SOC: 0.00076074
[[0.01729095]]
[[2.01797254]]
[[-1.33750045]]
[[-0.11532811]]
4.0
Current iteration: 5 ,SOC: 0.00073962
[[0.0210311]]
[[1.99902175]]
[[-1.31967512]]
[[-0.11411079]]
4.0
Current iteration: 6 ,SOC: 0.00071922
[[0.02479944]]
[[1.98029451]]
[[-1.30245218]]
[[-0.11286502]]
4.0
Current iteration: 7 ,SOC: 0.00069951
[[0.0285945]]
[[1.96178908]]
[[-1.28581972]]
[[-0.11159184]]
4.0
Current iteration: 8 ,SOC: 0.00068047
[[0.03241488]]
[[1.94350373]]
[[-1.26976604]]
[[-0.11029226]]
4.0
Current iteration: 9 ,SOC: 0.00066207
[[0.03625919]]
[[1.92543674]]
[[-1.25427957]]
[[-0.10896728]]
4.0
Cur

In [70]:
# without intercept
print(coefs, sigma)

[[ 0.54545844]
 [ 0.77468616]
 [-1.05534702]
 [-0.12943662]] -2.697


In [47]:
# with intercept
print(coefs, sigma)

[[-8.36391886]
 [ 0.74334431]
 [ 0.82117917]
 [-1.01138464]
 [ 0.24075693]] 2.697


### 4.2.3 Validation

#### Testing `Local_score2()` using `iwls()`

In [13]:
from spglm.iwls import iwls
from spglm.family import *

In [71]:
ln_Aij_test = np.log(calculate_Aij(X_new, -2.697)['Aij'].values.reshape((-1,1)))
X_new_test = np.hstack((np.log(X_new['Oi'].values).reshape((-1,1)), np.log(X_new['Dj'].values).reshape((-1,1)), np.log(X_new['Dij'].values).reshape((-1,1)), ln_Aij_test))
w = np.ones(72).reshape((-1,1))

coefs_test = iwls(flows.reshape((-1,1)), X_new_test, family = Poisson(), offset=w.reshape((-1, 1)), y_fix=np.ones(72), wi=w.reshape((-1, 1)))[0]
coefs_test

array([[ 0.54545844],
       [ 0.77468616],
       [-1.05534702],
       [-0.12943662]])

In [74]:
ln_Aij_test2 = np.log(calculate_Aij(X_new, -2)['Aij'].values.reshape((-1,1)))
X_new_test2 = np.hstack((np.log(X_new['Oi'].values).reshape((-1,1)), np.log(X_new['Dj'].values).reshape((-1,1)), np.log(X_new['Dij'].values).reshape((-1,1)), ln_Aij_test2))

y_new_test2 = np.exp(np.dot(X_new_test2, coefs_test))
coefs_tt, sigma_tt, eta_tt = local_score2(y_new_test2, X_new, True)

[[0.02048568]]
[[2.00991752]]
[[-1.43207156]]
[[-0.11169645]]
4.0
Current iteration: 1 ,SOC: 0.05179676
[[0.02369062]]
[[1.99140466]]
[[-1.41309927]]
[[-0.11063418]]
4.0
Current iteration: 2 ,SOC: 0.00078599
[[0.02692887]]
[[1.97310552]]
[[-1.39473564]]
[[-0.10954122]]
4.0
Current iteration: 3 ,SOC: 0.00076403
[[0.03019897]]
[[1.95501852]]
[[-1.37696887]]
[[-0.10841863]]
4.0
Current iteration: 4 ,SOC: 0.00074282
[[0.03349947]]
[[1.9371421]]
[[-1.35978738]]
[[-0.10726745]]
4.0
Current iteration: 5 ,SOC: 0.00072233
[[0.03682895]]
[[1.91947466]]
[[-1.34317973]]
[[-0.10608868]]
4.0
Current iteration: 6 ,SOC: 0.00070253
[[0.04018602]]
[[1.90201463]]
[[-1.32713466]]
[[-0.1048833]]
4.0
Current iteration: 7 ,SOC: 0.00068341
[[0.04356932]]
[[1.8847604]]
[[-1.31164109]]
[[-0.10365231]]
4.0
Current iteration: 8 ,SOC: 0.00066494
[[0.04697754]]
[[1.86771039]]
[[-1.2966881]]
[[-0.10239664]]
4.0
Current iteration: 9 ,SOC: 0.00064709
[[0.05040935]]
[[1.850863]]
[[-1.28226492]]
[[-0.10111723]]
4.0
Curr

In [16]:
#set sigma = -1
print(coefs_tt, sigma_tt)

[[ 0.56827656]
 [ 0.73970793]
 [-1.13195076]
 [-0.00914377]] 4.0


In [75]:
#set sigma = -2
print(coefs_tt, sigma_tt)

[[ 0.53962715]
 [ 0.76185382]
 [-1.08901723]
 [-0.08704269]] -2.697


In [73]:
#set sigma = -3
print(coefs_tt, sigma_tt)

[[ 0.61582423]
 [ 0.65131429]
 [-1.20892952]
 [ 0.06558467]] 2.697


#### Percent Misallocated & Sorensen Similarity Index

In [61]:
YYhat = np.hstack((flows.reshape((-1,1)), np.exp(eta)))
T = np.sum(flows.reshape((-1,1)))
PM = 50/T * np.sum(np.abs(np.diff(YYhat, axis = 1)))
PM

13.016869835566018

In [62]:
num = 2.0 * np.min(YYhat, axis=1)
den = flows + np.exp(eta).flatten()
N = 72
ssi = (1.0 / N) * (np.sum(num.reshape((-1, 1)) / den.reshape((-1, 1))))
ssi 

0.7656989415906664

#### Percent Misallocated of Unconstrained and Production-Constrained Model

In [24]:
#!pip install spint
import pysal
import spint
from spint import Gravity
from spint import Production 
from spint import Attraction 
from spint import Doubly

In [37]:
gravity = Gravity(flows, Oi, Wj, Dij, 'pow') 
print(gravity.params)

[-0.85755469  0.70317833  0.7376105  -1.05939577]


In [38]:
YYhat_unc = np.hstack((flows.reshape((-1,1)), gravity.yhat.reshape((-1,1))))
PM_unc = 50/T * np.sum(np.abs(np.diff(YYhat_unc, axis = 1)))
PM_unc

13.594037982969523

In [40]:
num_unc = 2.0 * np.min(YYhat_unc, axis=1)
den_unc = flows + gravity.yhat.flatten()
N = 72
ssi_unc = (1.0 / N) * (np.sum(num_unc.reshape((-1, 1)) / den_unc.reshape((-1, 1))))
ssi_unc 

0.7512053085272906

In [57]:
production = Production(flows, Origin, Wj, Dij, 'pow')
print(production.params)

[ 5.03090474  1.38000253  1.80970678  0.54871129  0.92185408  1.23154163
  0.63014444  0.92765753  0.51564951  0.73698043 -1.15536792]


In [39]:
YYhat_pd = np.hstack((flows.reshape((-1,1)), production.yhat.reshape((-1,1))))
PM_pd = 50/T * np.sum(np.abs(np.diff(YYhat_pd, axis = 1)))
PM_pd

11.45543019614379

In [41]:
num_pd = 2.0 * np.min(YYhat_pd, axis=1)
den_pd = flows + production.yhat.flatten()
N = 72
ssi_unc = (1.0 / N) * (np.sum(num_pd.reshape((-1, 1)) / den_pd.reshape((-1, 1))))
ssi_unc

0.7793448670313577